In [31]:
def pasar_text(nombre,X,y):
    f = open(nombre,'a')
    h,w = X.shape
    for i in range(h):
        linea = str(y[i]+1) + ' '
        for j in range(w):
            linea = linea + str(j+1) + ':' + str(X[i,j]) + ' '
        #print(linea)
        f.write(linea + '\n')
    f.close()

In [16]:
pasar_text('clasificacionTrayNoDensasSinAumentar', np.array(X), np.array(y))

# Clasificacion Usando Trayectorias

In [32]:
import os
import subprocess
import numpy as np
import pandas as pd

i = 1
home = os.getenv("HOME")
videos = home + '/' + 'Vídeos/'
videos_pk = frames = os.listdir(videos + 'pk/')
videos_std = frames = os.listdir(videos + 'std/')

X = []
y = []

for video in videos_pk:
    subprocess.call(['bash', 'sacar_trayectorias_(tipo).sh', video,'pk/'])
    Trayectories = pd.read_csv('trayectories', sep=' ', header=None)
    del Trayectories[34]
    Trayectories = Trayectories[abs(Trayectories[32]-Trayectories[2])>=2]
    Trayectories = Trayectories[abs(Trayectories[33]-Trayectories[3])>=2]
    data = Trayectories.values
    X.append(data)
    del Trayectories
    del data
    y.append(0)
    print('video: ' + str(i) + ' Procesado de: ' + str(len(videos_pk)+ len(videos_std)))
    i = i + 1

    
for video in videos_std:
    subprocess.call(['bash', 'sacar_trayectorias_(tipo).sh', video,'std/'])
    Trayectories = pd.read_csv('trayectories', sep=' ', header=None)
    del Trayectories[34]
    Trayectories = Trayectories[abs(Trayectories[32]-Trayectories[2])>=2]
    Trayectories = Trayectories[abs(Trayectories[33]-Trayectories[3])>=2]
    data = Trayectories.values
    X.append(data)
    del Trayectories
    del data
    y.append(1)         
    print('video: ' + str(i) + ' Procesado de: ' + str(len(videos_pk)+ len(videos_std)))
    i = i + 1
    

video: 1 Procesado de: 14
video: 2 Procesado de: 14
video: 3 Procesado de: 14
video: 4 Procesado de: 14
video: 5 Procesado de: 14
video: 6 Procesado de: 14
video: 7 Procesado de: 14
video: 8 Procesado de: 14
video: 9 Procesado de: 14
video: 10 Procesado de: 14
video: 11 Procesado de: 14
video: 12 Procesado de: 14
video: 13 Procesado de: 14
video: 14 Procesado de: 14


In [17]:
for i in range(14):
    tray = X[i]
    total = tray.shape[0]
    mitad = int(total/2)
    inicio = mitad - 1500
    final = mitad + 1500
    data = tray[inicio:final,:]
    data = data.reshape((1,-1))
    X[i] = data[0]

In [18]:
for i in range(14):
    print X[i].shape

(102000,)
(102000,)
(102000,)
(102000,)
(102000,)
(102000,)
(102000,)
(102000,)
(102000,)
(102000,)
(102000,)
(102000,)
(102000,)
(102000,)


In [19]:
import numpy as np
print('x.shape: ', np.array(X).shape, 'y.shape: ', np.array(y).shape)

('x.shape: ', (14, 102000), 'y.shape: ', (14,))


In [20]:
from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score

# Naive Gaussian

In [21]:
from sklearn.naive_bayes import GaussianNB
est = GaussianNB()
s = cross_val_score(est, X, y, cv=KFold(10, shuffle=True), scoring=make_scorer(accuracy_score))
print s
print "accuracy %.3f (+/- %.5f)"%(np.mean(s), np.std(s))

[1.  0.  0.5 0.  0.  1.  0.  1.  1.  0. ]
accuracy 0.450 (+/- 0.47170)


# DecisionTree

In [22]:
from sklearn.tree import DecisionTreeClassifier
est = DecisionTreeClassifier()
s = cross_val_score(est, X, y, cv=KFold(10, shuffle=True), scoring=make_scorer(accuracy_score))
print s
print "accuracy %.3f (+/- %.5f)"%(np.mean(s), np.std(s))

[0.5 1.  0.  1.  0.  0.  1.  0.  1.  0. ]
accuracy 0.450 (+/- 0.47170)


# Random Forest

In [23]:
from sklearn.ensemble import RandomForestClassifier
est = RandomForestClassifier()
s = cross_val_score(est, X, y, cv=KFold(10, shuffle=True), scoring=make_scorer(accuracy_score))
print s
print "accuracy %.3f (+/- %.5f)"%(np.mean(s), np.std(s))

[0.5 0.5 0.  0.5 0.  1.  0.  1.  0.  0. ]
accuracy 0.350 (+/- 0.39051)


# Support Vector Machine

In [24]:
from sklearn.svm import SVC
est = SVC()
s = cross_val_score(est, X, y, cv=KFold(4, shuffle=True), scoring=make_scorer(accuracy_score))
print s
print "accuracy %.3f (+/- %.5f)"%(np.mean(s), np.std(s))

[0.5        0.25       0.33333333 0.33333333]
accuracy 0.354 (+/- 0.09081)


# K-Nearest Neighbors

In [30]:
from sklearn.neighbors import KNeighborsClassifier
est = KNeighborsClassifier(n_neighbors = 9)
s = cross_val_score(est, X, y, cv=KFold(3, shuffle=True), scoring=make_scorer(accuracy_score))
print s
print "accuracy %.3f (+/- %.5f)"%(np.mean(s), np.std(s))

[0.4 0.4 1. ]
accuracy 0.600 (+/- 0.28284)


# Analisis de componentes principales 